In [1]:
import tomica
print(tomica.__file__)

/Users/zhangyangfa/rust-projects/tomica/tomica/__init__.py


In [7]:
import polars as pl

In [12]:
data = pl.Series([1,2,0,3])
data

""
i64
1
2
0
3


In [13]:
tomica.operators.ts_delay(data,1)

""
i64
null
1
2
0


In [14]:
tomica.operators.ts_delta(data,1)

""
i64
null
1
-2
3


In [15]:
tomica.operators.ts_returns(data,1)

""
i64
null
1
-1
null
